In [78]:
import requests
import os
from zipfile import ZipFile 
import json
from PyPDF2 import PdfReader


In [79]:
links = {
    "Mathematics": "https://s3.af-south-1.amazonaws.com/www.teachme2.com/matric-past-papers/2018-2023-Mathematics.zip",
    "Physical Sciences": "https://s3.af-south-1.amazonaws.com/www.teachme2.com/matric-past-papers/2018-2023-Physical-Sciences.zip",
    "Accounting": "https://s3.af-south-1.amazonaws.com/www.teachme2.com/matric-past-papers/2018-2023-Accounting.zip",
    "English": "https://s3.af-south-1.amazonaws.com/www.teachme2.com/matric-past-papers/2018-2023-English.zip"
}

In [80]:
response = requests.get("https://s3.af-south-1.amazonaws.com/www.teachme2.com/matric-past-papers/2018-2023-Mathematics.zip", stream =True)

In [81]:
url="https://s3.af-south-1.amazonaws.com/www.teachme2.com/matric-past-papers/2018-2023-Mathematics.zip"

In [82]:
local_filename = "2018-2023-Mathematics.zip"

In [83]:
# This function get's the files from 
def get_files(url, subject,path="."):
    file =f"{subject}.zip"


    file_path=os.path.join(path,file)
    

    try:
        response = requests.get(url,stream=True)
        response.raise_for_status()
        with open(file_path, "wb") as f:
            for chunk in response.iter_content(chunk_size=8192):  # Corrected here
                f.write(chunk)
                
        return print(f"sucessfuly downloaded and stored {subject}.zip file")
    except requests.exceptions.RequestException as e:
        print(f"Error observed while processing file {url}:{e}")
        return None
        




In [84]:
#Download zip files
folder_path = "."

for subject, url in links.items():
    file_path = get_files(url, subject, path=folder_path)
 

sucessfuly downloaded and stored Mathematics.zip file
sucessfuly downloaded and stored Physical Sciences.zip file
sucessfuly downloaded and stored Accounting.zip file
sucessfuly downloaded and stored English.zip file


In [85]:

#this finction extract zip files
def extract_files(files):
        
    for file in files:
        zip_file =f"{file}.zip"
        print(zip_file)
        # loading the temp.zip and creating a zip object 
        with ZipFile(zip_file, 'r') as zObject: 
        
        	# Extracting all the members of the zip 
        	# into a specific location. 
        	zObject.extractall(path=f"./extracts/{file}") 

In [86]:
# extract zip files

files =["Mathematics","Physical Sciences", "Accounting","English"]
extract_files(files)

Mathematics.zip
Physical Sciences.zip
Accounting.zip
English.zip


In [106]:
import re
import json
from PyPDF2 import PdfReader

def extract_questions(pdf_path):
    # Initialize a dictionary to store questions and answers
    questions_dict = {}

    # Read the PDF file
    with open(pdf_path, "rb") as pdf_file:
        pdf = PdfReader(pdf_file)
        text = ""
        
        # Extract text from each page
        for page_num, page in enumerate(pdf.pages):
            page_text = page.extract_text()
            text += page_text if page_text else f"Page {page_num + 1} has no text extracted.\n"
    
    # Define regex patterns for question numbers and sub-question numbers
    question_pattern = r"(\d+\.\d+(\.\d+)?)\s*([A-Za-z0-9'\".,;/\s\-]+)"
    sub_question_pattern = r"(\d+\.\d+\.\d+)\s*\.?\s*(.*)"

    # Split text into lines and process each line
    lines = text.splitlines()
    current_question = None
    current_sub_question = None

    for line in lines:
        # Check for main question or sub-question
        question_match = re.match(question_pattern, line)
        sub_question_match = re.match(sub_question_pattern, line)

        if question_match:
            # Capture the main question
            question_number = question_match.group(1)
            question_content = question_match.group(3).strip()

            # Start a new section in the dictionary
            current_question = question_number
            questions_dict[current_question] = {"content": question_content, "sub_questions": {}}

        elif sub_question_match:
            # Capture sub-question content
            sub_question_number = sub_question_match.group(1)
            sub_question_content = sub_question_match.group(2).strip()

            if current_question:
                # Add the sub-question to the appropriate question in the dictionary
                questions_dict[current_question]["sub_questions"][sub_question_number] = {
                    "content": sub_question_content
                }

    return questions_dict

def save_to_json(data, json_output_path):
    with open(json_output_path, "w", encoding="utf-8") as json_file:
        json.dump(data, json_file, ensure_ascii=False, indent=4)



In [110]:
import re
import json
from PyPDF2 import PdfReader

def extract_questions(pdf_path):
    # Initialize a dictionary to store questions and answers
    questions_dict = {}

    # Read the PDF file
    with open(pdf_path, "rb") as pdf_file:
        pdf = PdfReader(pdf_file)
        text = ""
        
        # Extract text from each page
        for page_num, page in enumerate(pdf.pages):
            page_text = page.extract_text()
            text += page_text if page_text else f"Page {page_num + 1} has no text extracted.\n"
    
    # Define regex patterns for question numbers and sub-question numbers
    question_pattern = r"(\d+\.\d+(\.\d+)?)\s*([A-Za-z0-9'\".,;/\s\-]+)"
    sub_question_pattern = r"(\d+\.\d+\.\d+)\s*\.?\s*(.*)"
    answer_pattern = r"([A-Za-z0-9'\-\s\.,:;]+)\n*(\d+[\.,\d]+)"  # Captures figures or additional details

    # Split text into lines and process each line
    lines = text.splitlines()
    current_question = None
    current_sub_question = None
    answer = ""

    for line in lines:
        # Check for main question or sub-question
        question_match = re.match(question_pattern, line)
        sub_question_match = re.match(sub_question_pattern, line)
        answer_match = re.match(answer_pattern, line)

        if question_match:
            # Capture the main question
            question_number = question_match.group(1)
            question_content = question_match.group(3).strip()

            # Start a new section in the dictionary
            current_question = question_number
            questions_dict[current_question] = {"content": question_content, "sub_questions": {}}
            answer = ""  # Reset answer after every new main question

        elif sub_question_match:
            # Capture sub-question content
            sub_question_number = sub_question_match.group(1)
            sub_question_content = sub_question_match.group(2).strip()

            if current_question:
                # Add the sub-question to the appropriate question in the dictionary
                if current_question not in questions_dict:
                    questions_dict[current_question] = {"content": "", "sub_questions": {}}
                
                questions_dict[current_question]["sub_questions"][sub_question_number] = {
                    "content": sub_question_content,
                    "answer": ""  # Placeholder for answer content
                }

        elif answer_match:
            # Capture additional details or answers after the question
            answer_content = answer_match.group(1).strip()
            figures = answer_match.group(2).strip()

            if current_question:
                # Add the answer content to the current sub-question
                if current_sub_question:
                    questions_dict[current_question]["sub_questions"][current_sub_question]["answer"] = {
                        "content": answer_content,
                        "figures": figures
                    }
                else:
                    questions_dict[current_question]["answer"] = {
                        "content": answer_content,
                        "figures": figures
                    }
    
    return questions_dict

def save_to_json(data, json_output_path):
    with open(json_output_path, "w", encoding="utf-8") as json_file:
        json.dump(data, json_file, ensure_ascii=False, indent=4)

# Example usage with your file paths
pdf_path = './extracts/Accounting/2018-Feb-March-Accounting-Memo-1-English.pdf'
json_output_path = 'output_file.json'

# Extract questions from the PDF and save to JSON
questions_data = extract_questions(pdf_path)
save_to_json(questions_data, json_output_path)

print(f"Questions extracted and saved to {json_output_path}")


Questions extracted and saved to output_file.json


In [111]:
# Example usage with your file paths
pdf_path = './extracts/Accounting/2018-Feb-March-Accounting-Paper-1-Answer-Book-English.pdf'
json_output_path = 'output_file.json'


# Extract questions from the PDF and save to JSON
questions_data = extract_questions(pdf_path)
save_to_json(questions_data, json_output_path)

print(f"Questions extracted and saved to {json_output_path}")

Questions extracted and saved to output_file.json


In [112]:
# Example usage with your file paths
pdf_path = './extracts/Accounting/2018-Feb-March-Accounting-Paper-1-Answer-Book-English.pdf'
json_output_path = 'output_file.json'

# Extract questions from the PDF and save to JSON
questions_data = extract_questions(pdf_path)
save_to_json(questions_data, json_output_path)

print(f"Questions extracted and saved to {json_output_path}")


Questions extracted and saved to output_file.json


In [116]:
import re
import json
from PyPDF2 import PdfReader

def extract_questions(pdf_path):
    # Initialize a dictionary to store questions and answers
    questions_dict = {}

    # Read the PDF file
    with open(pdf_path, "rb") as pdf_file:
        pdf = PdfReader(pdf_file)
        text = ""
        
        # Extract text from each page
        for page_num, page in enumerate(pdf.pages):
            page_text = page.extract_text()
            text += page_text if page_text else f"Page {page_num + 1} has no text extracted.\n"
    
    # Define regex patterns for question numbers and sub-question numbers
    question_pattern = r"(\d+\.\d+(\.\d+)?)\s*([A-Za-z0-9'\".,;/\s\-]+)"
    sub_question_pattern = r"(\d+\.\d+\.\d+)\s*\.?\s*(.*)"
    
    # Split text into lines and process each line
    lines = text.splitlines()
    current_question = None
    current_sub_question = None
    current_sub_question_content = ""
    
    for line in lines:
        # Check for main question or sub-question
        question_match = re.match(question_pattern, line)
        sub_question_match = re.match(sub_question_pattern, line)

        if question_match:
            # Capture the main question
            question_number = question_match.group(1)
            question_content = question_match.group(3).strip()

            # Start a new section in the dictionary
            current_question = question_number
            questions_dict[current_question] = {"content": question_content, "sub_questions": {}}
            current_sub_question_content = ""  # Reset content after every new main question

        elif sub_question_match:
            # Capture sub-question content (level 3: 1.1.1, 1.1.2 etc.)
            sub_question_number = sub_question_match.group(1)
            sub_question_content = sub_question_match.group(2).strip()

            if current_question:
                # Add the sub-question to the appropriate question in the dictionary
                if current_question not in questions_dict:
                    questions_dict[current_question] = {"content": "", "sub_questions": {}}
                
                # Start a new entry for sub-question (store it under sub-question number)
                questions_dict[current_question]["sub_questions"][sub_question_number] = {
                    "content": sub_question_content,
                    "details": ""  # Placeholder for any additional details, multiple-choice options, or further elaborations
                }
                current_sub_question = sub_question_number  # Track the current sub-question

        elif current_sub_question:
            # Handle multi-line answers or tables (append any continuation of the sub-question content)
            current_sub_question_content += line.strip() + "\n"  # Append the content to the details section
            
            # Once a line with a new question is detected, store the details
            if re.match(question_pattern, line):  # Detected a new main question (end of current details)
                if current_sub_question:
                    # Store the content under the current sub-question
                    questions_dict[current_question]["sub_questions"][current_sub_question]["details"] = current_sub_question_content.strip()
                current_sub_question_content = ""  # Reset details after storing it
                
            # Check if it’s the end of the document (if no new questions or sub-questions are detected)
            if line.strip() == "" and current_sub_question_content:
                # Store the final content for the current sub-question
                questions_dict[current_question]["sub_questions"][current_sub_question]["details"] = current_sub_question_content.strip()

    # Final step: Store remaining details if any sub-question content was pending
    if current_sub_question_content:
        questions_dict[current_question]["sub_questions"][current_sub_question]["details"] = current_sub_question_content.strip()
    
    return questions_dict

def save_to_json(data, json_output_path):
    with open(json_output_path, "w", encoding="utf-8") as json_file:
        json.dump(data, json_file, ensure_ascii=False, indent=4)

# Example usage with your file paths
pdf_path = './extracts/Accounting/2018-Feb-March-Accounting-Memo-1-English.pdf'
json_output_path = 'output_file.json'

# Extract questions from the PDF and save to JSON
questions_data = extract_questions(pdf_path)
save_to_json(questions_data, json_output_path)

print(f"Questions extracted and saved to {json_output_path}")


Questions extracted and saved to output_file.json


In [118]:
import json
from PyPDF2 import PdfReader

def pdf_to_json(pdf_path, json_output_path):
    try:
        # Open the PDF file
        with open(pdf_path, "rb") as pdf_file:
            pdf = PdfReader(pdf_file)
            text = ""
            
            # Extract text from each page of the PDF
            for page_num, page in enumerate(pdf.pages):
                page_text = page.extract_text()
                text += page_text if page_text else f"Page {page_num + 1} has no text extracted.\n"

        # Create a dictionary to store the extracted content
        text_object = {
            "content": text,
            "original_pdf_path": pdf_path
        }

        # Write the extracted text to a JSON file
        with open(json_output_path, "w", encoding="utf-8") as json_file:
            json.dump(text_object, json_file, ensure_ascii=False, indent=4)

        print(f"Extraction complete! JSON saved to: {json_output_path}")
    
    except Exception as e:
        print(f"Error: {e}")
        return

# Example usage
pdf_path = './extracts/Accounting/2018-Feb-March-Accounting-Memo-1-English.pdf'
json_output_path = 'output_file.json'
pdf_to_json(pdf_path, json_output_path)


Extraction complete! JSON saved to: output_file.json


In [132]:
import fitz  # PyMuPDF
import json
import re

def parse_pdf_with_pymupdf(file_path):
    # Read the PDF and extract text
    document_text = ""
    with fitz.open(file_path) as pdf:
        for page in pdf:
            document_text += page.get_text()

    # Split the text into lines for processing
    lines = document_text.split("\n")
    
    # Initialize variables
    json_structure = {}
    current_hierarchy = []
    current_content = []

    def set_nested_value(dictionary, keys, value):
        """Helper function to set a value in a nested dictionary."""
        for key in keys[:-1]:
            # Ensure each level is a dictionary
            dictionary = dictionary.setdefault(key, {})
        last_key = keys[-1]
        if last_key in dictionary:
            # Append to existing content if key exists
            if isinstance(dictionary[last_key], dict):
                raise ValueError("Expected a string but found a dictionary for the same level!")
            dictionary[last_key] += f"\n{value}"
        else:
            # Create new key with value
            dictionary[last_key] = value

    # Regex patterns for detecting levels (e.g., 1., 1.1, 1.1.1)
    level_pattern = re.compile(r'^(\d+(\.\d+)*)(?!\S)')

    for line in lines:
        match = level_pattern.match(line.strip())
        if match:
            # If a level is found, finalize the current content
            if current_hierarchy:
                set_nested_value(json_structure, current_hierarchy, "\n".join(current_content).strip())

            # Update the current hierarchy and reset content
            level = match.group(1).split(".")
            current_hierarchy = level
            current_content = [line.strip()]
        else:
            # Add line to current content if no new level is found
            current_content.append(line.strip())

    # Add the last collected content to the JSON structure
    if current_hierarchy:
        set_nested_value(json_structure, current_hierarchy, "\n".join(current_content).strip())

    return json_structure

# Example usage
if __name__ == "__main__":
    # Replace this with your PDF file path
    pdf_path = './extracts/Accounting/2018-Feb-March-Accounting-Memo-1-English.pdf'
    try:
        parsed_json = parse_pdf_with_pymupdf(pdf_path)

        # Save JSON to a file
        output_file = "parsed_document.json"
        with open(output_file, "w") as f:
            json.dump(parsed_json, f, indent=4)

        print(f"Parsed JSON has been saved to {output_file}")
    except Exception as e:
        print(f"An error occurred: {e}")


An error occurred: string indices must be integers, not 'str'


In [125]:
pdf_path = './extracts/Accounting/2018-Feb-March-Accounting-Memo-1-English.pdf'
json_output_path = 'output_file.json'

An error occurred: string indices must be integers, not 'str'


In [135]:
import fitz  # PyMuPDF
import json
import re

def parse_pdf_with_pymupdf(file_path):
    # Read the PDF and extract text
    document_text = ""
    with fitz.open(file_path) as pdf:
        for page in pdf:
            document_text += page.get_text()

    # Split the text into lines for processing
    lines = document_text.split("\n")
    
    # Initialize variables
    json_structure = {}
    current_hierarchy = []
    current_content = []

    def set_nested_value(dictionary, keys, value):
        """Helper function to set a value in a nested dictionary."""
        for key in keys[:-1]:
            # Ensure the structure is a dictionary
            if key not in dictionary or not isinstance(dictionary[key], dict):
                dictionary[key] = {}
            dictionary = dictionary[key]
        
        last_key = keys[-1]
        if last_key in dictionary:
            # If the key exists, decide how to handle it
            if isinstance(dictionary[last_key], dict):
                # Add content under a "content" key
                if "content" in dictionary[last_key]:
                    dictionary[last_key]["content"] += f"\n{value}"
                else:
                    dictionary[last_key]["content"] = value
            elif isinstance(dictionary[last_key], str):
                # Convert existing string to a dictionary with "content"
                dictionary[last_key] = {"content": dictionary[last_key] + f"\n{value}"}
            else:
                raise ValueError(f"Unexpected type for {last_key}: {type(dictionary[last_key])}")
        else:
            # Assign new value
            dictionary[last_key] = value

    # Regex patterns for detecting levels (e.g., 1., 1.1, 1.1.1)
    level_pattern = re.compile(r'^(\d+(\.\d+)*)(?!\S)')

    for line in lines:
        match = level_pattern.match(line.strip())
        if match:
            # If a level is found, finalize the current content
            if current_hierarchy:
                set_nested_value(json_structure, current_hierarchy, "\n".join(current_content).strip())

            # Update the current hierarchy and reset content
            level = match.group(1).split(".")
            current_hierarchy = level
            current_content = [line.strip()]
        else:
            # Add line to current content if no new level is found
            current_content.append(line.strip())

    # Add the last collected content to the JSON structure
    if current_hierarchy:
        set_nested_value(json_structure, current_hierarchy, "\n".join(current_content).strip())

    return json_structure

# Example usage
if __name__ == "__main__":
    # Replace this with your PDF file path
    pdf_path = './extracts/Accounting/2018-Feb-March-Accounting-Memo-1-English.pdf'
    try:
        parsed_json = parse_pdf_with_pymupdf(pdf_path)

        # Save JSON to a file
        output_file = "parsed_document.json"
        with open(output_file, "w") as f:
            json.dump(parsed_json, f, indent=4)

        print(f"Parsed JSON has been saved to {output_file}")
    except Exception as e:
        print(f"An error occurred: {e}")


Parsed JSON has been saved to parsed_document.json
